In [1]:
import torch
from peft import LoraConfig, PeftModel
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline)

In [2]:
model_identifier="aboonaji/llama2finetune-v2"

In [3]:
enable_4bit=True
compute_dtype_bnb="float16"
quant_type_bnb="nf4"
double_quant_flag=False

In [4]:
device_assignment = {"": 0}

In [5]:
dtype_computation = getattr(torch, compute_dtype_bnb)
bnb_setup = BitsAndBytesConfig(load_in_4bit = enable_4bit,
                               bnb_4bit_quant_type = quant_type_bnb,
                               bnb_4bit_use_double_quant = double_quant_flag,
                               bnb_4bit_compute_dtype = dtype_computation)

In [6]:
llama_model = AutoModelForCausalLM.from_pretrained(model_identifier, quantization_config = bnb_setup, device_map = device_assignment)
llama_model.config.use_case = False
llama_model.config.pretraining_tp = 1

c:\ProgramData\miniconda3\envs\llmenv\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

c:\ProgramData\miniconda3\envs\llmenv\lib\site-packages\transformers\modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, 

In [7]:
llama_tokenizer = AutoTokenizer.from_pretrained(model_identifier, trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

In [8]:
adapters_path = 'D:/SUPER DATA SCIENCE/ARTIFICIAL INTELLIGENCE/AI_DOCTER'
model = PeftModel.from_pretrained(llama_model, adapters_path)

c:\ProgramData\miniconda3\envs\llmenv\lib\site-packages\peft\peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


In [9]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (

In [10]:
def generate_text(prompt, model, tokenizer, max_length=400):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [11]:
user_prompt = "treatment for cough and cold"
generated_text = generate_text(f"<s>[INST] {user_prompt} [/INST]", model, llama_tokenizer)
print(generated_text)

c:\ProgramData\miniconda3\envs\llmenv\lib\site-packages\transformers\generation\utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
c:\ProgramData\miniconda3\envs\llmenv\lib\site-packages\transformers\generation\utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[INST] treatment for cough and cold [/INST]  There are several treatments for cough and cold, depending on the severity and duration of the symptoms. Here are some common treatments for cough and cold:

1. Over-the-counter (OTC) medications:
	* Cough suppressants (e.g., dextromethorphan) to relieve coughing
	* Expectorants (e.g., guaifenesin) to loosen and clear mucus from the airways
	* Antihistamines (e.g., diphenhydramine) to reduce nasal congestion and sneezing
	* Decongestants (e.g., pseudoephedrine) to reduce nasal congestion
	* Pain relievers (e.g., acetaminophen or ibuprofen) to reduce fever and relieve headache or body aches
2. Prescription medications:
	* Antiviral medications (e.g., oseltamivir, zanamivir) to treat viral infections
	* Bronchodilators (e.g., albuterol, levalbuterol) to relieve wheezing and shortness of breath
	* Corticosteroids (e.g., prednisone) to reduce inflammation and swelling in the airways
	* Antibiotics (e.g., amoxicillin, azithromycin) to treat bacte